### Modelado

En este notebook utilizaremos los conjuntos que hemos inspeccionado, adecuado y preprocesado para entrenar modelos que nos ayuden a analizar los mensajes de texto y detectar si se trata de **spam** o no.

El siguiente script está dividido en los siguientes bloques:

- **BLOQUE A**: carga de datos preprocesados.
- **BLOQUE B**: entrenamiento y inferencia con distintos modelos de ML.
- **BLOQUE C**: entrenamiento y inferencia con una red neuronal (librería Keras).
- **BLOQUE D**: transfer learning con un modelo preentrenado.

In [ ]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report


from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer

### BLOQUE A: Carga de datos
Antes de comenzar, cargaremos los datos que han sido adecuados en nuestra fase anterior de preprocesado

In [ ]:
# Carga textos vectorizados
with open('../data/x_train_vec.pkl', 'rb') as f:
    X_train_vec = pickle.load(f)

with open('../data/x_test_vec.pkl', 'rb') as f:
    X_test_vec = pickle.load(f)

In [ ]:
# Carga conjuntos de las etiquetas
with open('../data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('../data/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

### BLOQUE B: Entrenamiento de distinto modelos de ML

#### Regresión logistica

La regresión logística es un modelo estadístico para estudiar las relaciones entre un conjunto de variables cualitativas Xi y una variable cualitativa Y. Utilizando la función sigmoide, asigna valores entre 0 y 1, facilitando la predicción de categorías, como positivo o negativos, spam o no spam, en aplicaciones prácticas.

Más información acerca de este modelo [aquí](https://cienciadedatos.net/documentos/py17-regresion-logistica-python.html).

In [ ]:
# Creamos el modelo
log_model = ???()

In [ ]:
# Entrenamiento o ajuste del modelo con los datos de entrenamiento
log_model.???(X_train_vec, y_train)

In [ ]:
# Predecimos sobre los datos de entrenamiento
y_pred_train = log_model.???(X_train_vec)

# Mostramos el "classification report"
print('Resultados conjunto de entrenamiento:\n')
print(classification_report(???, ???))

In [ ]:
# Predecimos sobre los datos de test
y_pred_test = log_model.???(X_test_vec)

# Mostramos el "classification report" y "accuracy"
accuracy = accuracy_score(???, ???)
print('Resultados conjunto de test:\n')
print(f'Accuracy: {accuracy:.2f}\n')
print(classification_report(???, ???))

#### Gradient boosting

¿Qué es Boosting?

Boosting es un meta-algoritmo de aprendizaje automático que reduce el sesgo y la varianza en un contexto de aprendizaje supervisado. Consiste en combinar los resultados de varios clasificadores débiles para obtener un clasificador robusto. Cuando se añaden estos clasificadores débiles, se hace de modo que éstos tengan diferente peso en función de la exactitud de sus predicciones. Tras añadir un clasificador débil, los datos cambian su estructura de pesos: los casos mal clasificados ganan peso y los que son clasificados correctamente pierden peso.

**Gradient Boosting (GB)** o Potenciación del gradiente consiste en plantear el problema como una optimización numérica en el que el objetivo es minimizar una función de coste añadiendo clasificadores débiles mediante el descenso del gradiente. Involucra tres elementos:

- La **función de coste** a optimizar: depende del tipo de problema a resolver.
- Un **clasificador débil** para hacer las predicciones: por lo general se usan árboles de decisión.
- Un **modelo que añade (ensambla) los clasificadores débiles para minimizar la función de coste**: se usa el descenso del gradiente para minimizar el coste al añadir árboles.

Los hiperparámetros más importantes que intervienen en este algoritmo (aunque no todos) son:

- **learning_rate**: determina el impacto de cada árbol en la salida final. Se parte de una estimación inicial que se va actualizando con la salida de cada árbol. Es el parámetro que controla la magnitud de las actualizaciones.
- **n_estimators**: número de clasificadores débiles a utilizar.

Como en este caso utilizaremos árboles de decisión como clasificadores débiles a ensamblar, también debemos tener en cuenta los hiperparámetros que afectan a esta clase de modelos. En este caso:

- **max_depth**: profundidad máxima del árbol.

Más información sobre el modelo que se utiliza en este ejemplo y de sus parámetros [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html).

In [ ]:
# Creamos el modelo introduciendo los valores de los parámetros:
gb_clf = ???(n_estimators=150, learning_rate=0.2, max_depth=3, random_state=0)

In [ ]:
# Entrenamiento o ajuste del modelo con los datos de entrenamiento
gb_clf.???(X_train_vec, y_train)

In [ ]:
# Predecimos sobre los datos de entrenamiento
pred_train = gb_clf.???(X_train_vec)

# Mostramos el "classification report"
print('Resultados conjunto de entrenamiento:\n')
print(???(y_train, pred_train))

In [ ]:
# Predecimos sobre los datos de test
pred_test = gb_clf.???(X_test_vec)

# Mostramos el "classification report" y "accuracy"
accuracy = accuracy_score(???, ???)
print('Resultados conjunto de test:\n')
print(f'Accuracy: {accuracy:.2f}\n')
print(???(y_test, pred_test))

### BLOQUE C: entrenamiento y inferencia con una red neuronal (librería Keras)

Las redes neuronales artificiales son una rama de la Inteligencia Artificial que se basa en la simulación de la estructura y funcionamiento del cerebro humano para procesar información.

Consisten en una serie de nodos interconectados que reciben información, la procesan y producen una salida. Dichos nodos, conocidos como neuronas artificiales, pueden ser ajustados para optimizar la salida de la red, lo que permite que la red aprenda y se adapte a diferentes tipos de entradas.

*Curiosidad: cuando una red neuronal tiene simplemente más de tres capas, nace el conocido como **Deep Learning** o aprendizaje profundo.*

Más información sobre este tipo de modelos [aquí](https://openwebinars.net/blog/que-son-las-redes-neuronales-y-sus-aplicaciones/).

En este caso, entrenaremos la red para que también obtenga una representación de los textos analizados de manera directa, sin necesidad de utilizar el modelo de vectorizado TF-IDF antes calculado.

In [ ]:
# Cargar el dataset (ajusta la ruta según el archivo que tengas)
df = pd.read_csv(???)

# Verifica el dataset
df.head()

In [ ]:
# Preprocesamiento del texto
X = df[???]  # Supón que la columna 'text' contiene los mensajes
y = df[???]  # 'label' debe ser spam/no spam

In [ ]:
# Convertir las etiquetas 'spam'/'ham' a números
encoder = LabelEncoder()
y = encoder.???(y)

In [ ]:
# ¿Que valores asume la variale target transformada? 
y

In [ ]:
# ¿A qué corresponden los nuevos valores?
df[???].values

In [ ]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = ???(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Tokenización y padding para convertir el texto en secuencias de enteros
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.???(X_train)

X_train_seq = tokenizer.???(X_train)
X_test_seq = tokenizer.???(X_test)

In [ ]:
# ¿Como son transformados los textos con Tokenizer?
X_train_seq[0]

In [ ]:
# Padding para asegurar que todas las secuencias tengan la misma longitud
X_train_pad = ???(X_train_seq, padding='post', maxlen=100)
X_test_pad = ???(X_test_seq, padding='post', maxlen=100)

Estructura de la red neuronal:
1. Capa de Embedding: 
    - input_dim=10000: Esto se refiere al tamaño del vocabulario (número total de palabras). 

    - output_dim=64: Este especifica la dimensión de los embeddings. En lugar de representar cada palabra con un valor binario o en un vector disperso (como ocurre con TF-IDF), las palabras se representarán por un vector denso de 64 dimensiones. 

    - input_length=100: se define la longitud máxima de las secuencias de entrada. Cada entrada de texto se convierte en una secuencia de índices de palabras (tokens), y si la longitud de una secuencia es menor que 100, se rellenará con ceros. Si es mayor, se truncará.
2. Capa de Pooling: Esta capa realiza un *pooling global promedio* sobre las secuencias de embeddings. Dado que las entradas son secuencias de longitud 100, esta capa tomará el promedio de los vectores de embeddings de las 100 palabras (tokens) en la secuencia, lo que reduce la representación de la secuencia de una matriz de 100x64 a un solo vector de 64 dimensiones

3. Capa densa (Fully connected): Esta capa toma la salida del pooling (un vector de 64 dimensiones) y lo pasa a través de una **capa densa con 64 neuronas** y la **función de activación ReLU**. Esto ayuda a la red a aprender características más complejas del texto procesado.

4. Capa de salida: esta capa tiene una sola neurona con la **función de activación Sigmoid**, que es adecuada para clasificación binaria (en este caso, para clasificar como spam o no spam). La salida de esta capa será un valor entre 0 y 1, que puedes interpretar como la probabilidad de que un mensaje sea spam.



In [ ]:
# Modelo simple de red neuronal para clasificación de texto
nn_simple = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=???, output_dim=???, input_length=???),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(???, activation=???),
    tf.keras.layers.Dense(???, activation=???)  # Para clasificación binaria (spam/no spam)
])

In [ ]:
# Compilar el modelo
nn_simple.???(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
nn_simple.???(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

In [ ]:
# Predecimos sobre los datos de test
pred_test = nn_simple.???(X_test_pad)

In [ ]:
# ¿Qué contiene pred_test? ¿Qué son?
pred_test

In [ ]:
# Convertir las predicciones a clases (spam o no spam)
# La salida será un valor entre 0 y 1 (probabilidad de spam)
predicted_labels = (pred_test > ???).astype(int)  # Clasifica como spam (1) si la probabilidad > 0.5, sino no spam (0)
predicted_labels

In [ ]:
# Mostramos el "classification report" y "accuracy"
accuracy = accuracy_score(y_test, predicted_labels)
print('Resultados conjunto de test:\n')
print(f'Accuracy: {accuracy:.2f}\n')
print(classification_report(y_test, predicted_labels))

### Transfer Learning con un modelo preentrenado

**BERT** (Bidirectional Encoder Representations from Transformers) es una técnica basada en redes neuronales para el pre-entrenamiento del procesamiento del lenguaje natural (NLP) desarrollada por Google. Se trata de un codificador que obtiene representaciones bidireccionales, lo que significa que puede entender el significado de una palabra en relación con las palabras que la rodean. Esto permite al modelo aprender a **interpretar el lenguaje en general y ser utilizado como modelo base**, de manera que, en caso de querer especializarlo en una tarea en particular (detección de spams, por ejemplo), solo es necesario añadir unas capas adicionales al mismo.

El modelo original de BERT se entrenó utilizando dos grandes conjuntos de datos en lengua inglesa: BookCorpus y Wikipedia en inglés. 

*Curiosidad: actualmente, Google utiliza BERT en su motor de búsqueda para perfeccionar la comprensión de las búsquedas de los usuarios.*

Por su parte, **DistilBERT** (Distilled version of BERT) es una versión más ligera, rápida y menos costosa del modelo BERT, como su propio nombre indica.

Más información acerca de BERT [aquí](https://huggingface.co/blog/bert-101), y de DistilBERT [aquí](https://huggingface.co/docs/transformers/model_doc/distilbert) y [aquí](https://medium.com/huggingface/distilbert-8cf3380435b5).

In [ ]:
# Cargar el tokenizer de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
# Tokenización de los textos
X_train_enc = tokenizer(list(X_train[:100]), padding=True, truncation=True, max_length=100, return_tensors='tf') # Usamos 100 observaciones para poder ejecutarlo en local
X_test_enc = tokenizer(list(X_test), padding=True, truncation=True, max_length=100, return_tensors='tf')

# Cargar el modelo preentrenado DistilBERT para clasificación de texto
model_bert = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Compilar el modelo
model_bert.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento con transfer learning
model_bert.fit(X_train_enc['input_ids'], y_train[:100], epochs=3, batch_size=128, validation_data=(X_test_enc['input_ids'], y_test))

In [ ]:
# Evaluamos sobre el conjunto de test y vemos métrica
loss, accuracy = model_bert.evaluate(X_test_enc['input_ids'], y_test)
print(f"Accuracy DistilBERT model: {accuracy}")